# Red Wine Quality Prediction 

Authors (Alphabetical by Last Name): Nicole Bidwell, Ruocong Sun, Alysen Townsley, Hongyang Zhang

## Summary 

In this project our group seeks to use machine learning algorithms to predict wine quality (scale of 0 to 10) using physiochemical properties of the liquid. We use a train-test split and cross-validation to simulate the model encountering unseen data. We use and tune the parameters of several classification models: logistic regression, decision tree, kNN, and SVM RBF to see which one has the highest accuracy, and then deploy the winner onto the test set. The final test set accuracy is around 62 percent. Depending on the standard, this can be decent or poor. However, a more important note is that for the really extreme quality ones (below 5 or above 6), the model was unable to identify quite a few of them correctly, suggesting that it is not very robust to outliers. We include a final discussion section on some of the potential causes for this performance as well as proposed solutions for any future analysis.

## Introduction 

Red wines have a long history that can be traced all the way back to the ancient Greeks. Today, they are more accessible to an average person than ever and the entire industry is estimated to be worth around 109.5 billion USD (The Business Research Company). Despite its ubiquity, most people can barely tell the difference between a good and a bad wine, to the point where we need trained professionals (sommeliers) to understand the difference. In this project, we seek to use machine learning algorithms to predict the quality of the wine based on the physiochemical properties of the liquid.  This model, if effective, could allow manufactures and suppliers to have a more robust understanding of the wine quality based on measurable properties.

## Methods & Results

In [14]:
import pandas as pd 
import numpy as np
import sys
import os

sys.path.append('..')
from src.model_hyperparam_tuning1 import model_hyperparam_tuning

from hashlib import sha1
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.dummy import DummyClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    cross_validate,
    train_test_split,
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import altair as alt
from sklearn.linear_model import Ridge
from sklearn.metrics import ConfusionMatrixDisplay 

### EDA
#### Dataset Description
The dataset is the "winequality-red.csv" file from the <a href=https://archive.ics.uci.edu/dataset/186/wine+quality>UC Irvine Machine Learning Repository</a>, which was originally referenced from <a href=http://www3.dsi.uminho.pt/pcortez/wine/>Decision Support Systems, Elsevier</a>. The dataset contains physiochemical proprties (features) of red vinho verde wine samples from the north of Portugal, along with an associated wine quality score from 0 (worst) to 10 (best). 

In [2]:
# Import the dataset:
df = pd.read_csv('../data/winequality-red.csv', sep = ';')

# Visualize the first 5 rows:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


<sub>Table 1: First five rows of the red wine dataframe.<sub>

There are 11 feature columns representing physiochemical characteristics of the wines, such as fixed acidity, residual sugar, chlorides, density, etc. There are 1598 rows or observations in the dataset, with no missing values. The target is the quality column which is listed as a set of ordinal values from 3 to 8, although they could go as low as 0 or as high as 10 (this data set does not contain observations across the entire range). Most observations have an "average" quality of 5 or 6, with fewer below a score of 5 or above a score of 6.

#### Visualization

We first observe the distribution of the features using their statistical summaries and a histogram. We can see that the majority of features have a skewed distribution, with many containing outliers. Volatile acidity, residual sugar, chlorides, free sulfur dioxide, total sulfur dioxide, and sulphates all have very extreme outliers.

In [3]:
# Features 
feature_df = df.drop('quality', axis=1)

feature_names = list(feature_df.columns)

alt.Chart(feature_df).mark_bar().encode(
     alt.X(alt.repeat()).type('quantitative').bin(maxbins=40),
     y='count()',
).properties(
    width=250,
    height=250
).repeat(
    feature_names, 
    columns=3
).properties(
    title = "Figure 1: Histograms showing the distrbution of each feature in the red wine dataframe."
).configure_title(
    orient='bottom', 
    anchor = 'middle'
)

alt.RepeatChart(...)

### Model Training
#### Model Selection and Hyperparameter Tuning

Our method for model selection involves using 5-fold cross-validation and hyperparameter tuning on several models: logistic regression, decision tree, kNN and SVM RBF. We use validation accuracy as our metric. Below we first use a dummy classifier to establish the baseline.

In [4]:
X = df.drop(columns = ['quality'])
y = df['quality']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 522)

In [5]:
#Baseline
baseline = DummyClassifier(random_state=522)
pd.DataFrame(cross_validate(baseline, X_train, y_train, return_train_score=True))

,fit_time,score_time,test_score,train_score
0,0.001000,0.000999,0.441964,0.440223
1,0.001001,0.000000,0.437500,0.441341
2,0.000000,0.001000,0.437500,0.441341
3,0.001000,0.000000,0.441964,0.440223
4,0.001000,0.000000,0.443946,0.439732


<sub>Table 2: Cross valdidation results for the Dummy Classifier baseline model.<sub>

In [6]:
np.mean(pd.DataFrame(cross_validate(baseline, X_train, y_train, return_train_score=True)), axis=0)

fit_time       0.000273
score_time     0.000400
test_score     0.440575
train_score    0.440572
dtype: float64

As we can see, the baseline obtains an accuracy of around 44 percent. We now use cross cross validation paired with hyperparameter tuning to identify a model that performs the best. 

In [19]:
lr_grid_search = model_hyperparam_tuning(X_train, y_train, 'logistic', {'model__C': [0.001, 0.01, 0.1, 1.0, 10, 100, 1000],
                                                                        'model__class_weight': ['balanced', None]})

ValueError: Invalid parameter 'model' for estimator LogisticRegression(random_state=522). Valid parameters are: ['C', 'class_weight', 'dual', 'fit_intercept', 'intercept_scaling', 'l1_ratio', 'max_iter', 'multi_class', 'n_jobs', 'penalty', 'random_state', 'solver', 'tol', 'verbose', 'warm_start'].

In [16]:
lr_df = (pd.DataFrame(lr_grid_search.cv_results_)[
    [
        "mean_test_score",
        "mean_train_score",
        "param_model__C",
        "param_model__class_weight",
        "mean_fit_time",
        "rank_test_score",
    ]
].set_index("rank_test_score").sort_index())
lr_df.head(1)

,mean_test_score,mean_train_score,param_model__C,param_model__class_weight,mean_fit_time
rank_test_score,,,,,
1,0.586275,0.592495,0.1,None,0.025788


<sub>Table 3: Grid search results for logistic regression hyperparameter tuning.<sub>

In [ ]:
dt_grid_search = model_hyperparam_tuning(X_train, y_train, 'decision_tree', {'criterion': ['gini', 'entropy'],
                                                                             'max_depth': 2 ** np.arange(8),
                                                                             'class_weight': ['balanced', None]})

In [ ]:
dt_df = (pd.DataFrame(dt_grid_search.cv_results_)[
    [
        "mean_test_score",
        "mean_train_score",
        "param_model__criterion",
        "param_model__max_depth",
        "param_model__class_weight",
        "mean_fit_time",
        "rank_test_score",
    ]
].set_index("rank_test_score").sort_index())
dt_df.head(1)

<sub>Table 4: Grid search results for decision tree hyperparameter tuning.<sub>

In [ ]:
knn_grid_search = model_hyperparam_tuning(X_train, y_train, 'knn', {'n_neighbors': [1, 2, 3, 4, 5, 6]})

In [ ]:
knn_df = (pd.DataFrame(knn_grid_search.cv_results_)[
    [
        "mean_test_score",
        "mean_train_score",
        "param_model__n_neighbors",
        "mean_fit_time",
        "rank_test_score",
    ]
].set_index("rank_test_score").sort_index())
knn_df.head(1)

<sub>Table 5: Grid search results for KNN hyperparameter tuning.<sub>

In [ ]:
svc_grid_search = model_hyperparam_tuning(X_train, y_train, 'svc', {'C': [0.001, 0.01, 0.1, 1.0, 10, 100, 1000],
                                                                    'gamma': [0.001, 0.01, 0.1, 1.0, 10, 100, 1000],
                                                                    'class_weight': ['balanced', None]})

In [ ]:
svc_df = (pd.DataFrame(svc_grid_search.cv_results_)[
    [
        "mean_test_score",
        "mean_train_score",
        "param_model__C",
        "param_model__gamma",
        "param_model__class_weight",
        "mean_fit_time",
        "rank_test_score",
    ]
].set_index("rank_test_score").sort_index())
svc_df.head(1)

<sub>Table 6: Grid search results for SVC hyperparameter tuning.<sub>

We see that logistic regression has a best validation score of 58.6 percent; decision tree is 59.3 percent; kNN is 59.8 percent, and RBF SVM is 61.3 percent. As a result, we will use the tuned RBF SVM as our model on the test set.

#### Test Set Deployment

In [ ]:
best_pipe = make_pipeline(StandardScaler(), SVC(C=1000, gamma=0.01, class_weight=None))
best_pipe.fit(X_train, y_train)
best_pipe.score(X_test, y_test)

There is a slight improvement in accuracy with the test set. We want to further probe into its performance by looking at the confusion matrix.

In [ ]:
ConfusionMatrixDisplay.from_estimator(
    best_pipe,
    X_test,
    y_test,
    values_format="d",  # Show the full number 59,700 instead of 6e+04,
)
plt.title('Figure 2: Confusion matrix of the SVC model performance on the test data.', y = -0.2, loc='center')

For the really mediocre wines (5 and 6), the model can predict most of them correctly, but the model fails to predict a large proportion of extreme ones correctly, suggesting that the model is not too robust against outliers.

## Discussion 

In this project, we built several machine learning classification models seeking to predict the wine quality based on the physiochemical properties of the liquid. By trying out different models with different hyperparameters, we have found that for our data set, the best performing model is RBF SVM. However, despite being the best, the accuracy is only ~62.3 percent. Depending on the situation this can be poor or decent. More importantly, the algorithm seems to not be able to identify the outliers precisely, and in the case where people want to be able to find really good or bad wines, this model's performance would not be able to meet people's expectations. Our group's discussion has concluded that there might be several factors leading to this phenomenon:

### High correlations:

In [ ]:
# Features 
feature_df = df.drop('quality', axis=1)

feature_names = list(feature_df.columns)

feature_df.corr(numeric_only=True).style.background_gradient('PiYG')

<sub>Table 7: Correlation matrix for all red wine physiochemical features in the dataframe.<sub>

Several variables in the data set appear to have a substantial amount of correlation (in the range of 0.6) and this collinearity could have potentially caused problems with some of our models. Given this and the high dimensionality, we could have implemented a dimensionality reduction algorithm (such as PCA) to reduce the number of features and therefore eliminate some of the collinearity.

### Potential Interactions:
In our logistic regression model we did not take any of the potential interaction into the account. With this many qualities it is possible that some of the features affect the effect of others.

### Problem Formulation:
The response variable could be treated as a number instead and an approach of regression question could have better captured the nature of our problem and produced a better model. Additionally, due to the limited scope of our data set (no observation below 3 or above 8), a classification model trained on this data set would not be able to identify any observation outside of the scope correctly. A regression algorithm is more immune to this kind of problem. 

### Infeasibility of the Problem
Despite the potential improvements we have identified (or not) for our project, there still exists a possibility that even with all these improvements, the accuracy would not improve that much. And that is not due to the incorrect setup for the analyses, but rather the fact that some of the underlying uncontrollable factors in the process of wine making simply makes it impossible to detect patterns for really good or bad wines, and their qualities can only determined by actually tasting rather than prediction using numerical representations of some of its properties. However, among all the possible problems we have identified, this is the only one where we have zero proposed solutions for.

## References 

UCI Machine Learning Repository. (2017). https://archive.ics.uci.edu/

Red Wine Market Size, Industry Share, Trends, Analysis Forecast. (2023, January). https://www.thebusinessresearchcompany.com/report/red-wine-global-market-report#:~:text=The%20global%20red%20wine%20market%20size%20grew%20from%20%24102.97%20billion,least%20in%20the%20short%20term

Pramoditha, R. (2022, January 6). How do you apply PCA to Logistic Regression to remove Multicollinearity? Medium. https://towardsdatascience.com/how-do-you-apply-pca-to-logistic-regression-to-remove-multicollinearity-10b7f8e89f9b#:~:text=PCA%20(Principal%20Component%20Analysis)%20takes,effectively%20eliminate%20multicollinearity%20between%20features.

Deciphering interactions in logistic regression. (n.d.). https://stats.oarc.ucla.edu/stata/seminars/deciphering-interactions-in-logistic-regression/#:~:text=Logistic%20interactions%20are%20a%20complex%20concept&text=But%20in%20logistic%20regression%20interaction,can%20make%20a%20big%20difference.